In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [2]:
import time
from pathlib import Path
from tqdm import tqdm

In [3]:
from manage_images import check_images_for_doubles, delete_doubles, add_id_db, is_id_in_db
from process_whatsapp_web import process_message

In [4]:
IMG_DIR_PATH = Path('/Users/frederic/Documents/GitHub/dynamicSlideshow/server/images_prod/whatsapp')
DB_PATH = Path('./21_db/image_list.txt')

In [5]:
driver = Firefox(executable_path='./40_driver/geckodriver')

/Users/frederic/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [6]:
driver.get('https://web.whatsapp.com/')

In [7]:
0 / 0

ZeroDivisionError: division by zero

In [8]:
def process_message( message, driver, db_path ):
    img_id = message.get_attribute("data-id")
    
    if is_id_in_db( img_id, db_path ) == False:
        try:
            imgs = message.find_elements(
                by=By.CSS_SELECTOR, 
                value='div[data-testid="image-thumb"]'
            )
            
            #print( len(imgs) )
        except:
            imgs = []

        for img in imgs:
            img.click()
            time.sleep(0.5)

            try:
                download_button = driver.find_element(
                    by=By.CSS_SELECTOR,
                    value='div[title="Download"]'
                )
                download_button.click()
            except:
                menu_button = driver.find_element(
                    by=By.CSS_SELECTOR,
                    value='div[title="Menü"]'
                )
                menu_button.click()
                
                download_button = driver.find_element(
                    by=By.CSS_SELECTOR,
                    value='div[aria-label="Download"]'
                )
                download_button.click()
                

            time.sleep(0.5)

            close_button = driver.find_element(
                by=By.CSS_SELECTOR,
                value='div[title="Schließen"]'
            )

            close_button.click()
            time.sleep(1.5)
            
        return img_id
    else:
        raise Exception("img already in db")

In [9]:
while True:
    # Chats lesen
    chats = driver.find_elements(by=By.XPATH, value='//div[@data-testid="chat-list"]/div/div')

    # Bilder laden
    for chat in chats:
        chat.click()

        messages = driver.find_elements(by=By.CSS_SELECTOR, value="div.message-in")

        for message in tqdm(messages):
            try:
                img_id = process_message( message, driver, DB_PATH )
                add_id_db( img_id, DB_PATH )
            except Exception as e:
                pass

    hash_table = check_images_for_doubles( IMG_DIR_PATH )
    delete_doubles(hash_table)
    
    time.sleep(30)

100%|██████████████████████████████████████████| 14/14 [00:00<00:00, 154.31it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 147.86it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 174.87it/s]


100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 145.83it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 145.80it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 157.84it/s]


100%|████████████████████████████████████████████| 6/6 [00:00<00:00, 143.26it/s]


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 158.34it/s]


KeyboardInterrupt: 

In [ ]:
driver.close()